# tAPP 5: Obtaining, Cleaning and Validating Baby Names Data with Bash One-Liners, Part II

**Author:** [Your Name]

**Team:** [Your Team Name]

**Date:** Mar 3, 2025

**Course:** DSC 011 Sp25

## The Goal and Task of This Assignment

The Goal of today's Lecture is to continue to demonstrate using UNIX Bash command pipelines and Perl one-liners to validate and statistically explore the cleaned and integrated master data-file we created and started validating last time in tAPP 4 called `baby_clean.csv`, a small (2800 row by 6 column) data-set of top-200 U.S. baby names from the US Census data and downloaded from the Social Security Administration. 
 


**Your Task: No matter whether you already worked tAPP4 and have the validated file `baby_clean.csv` or you missed tAPP4 and you do not have this file, run all of the code chunks with shift-enter in this notebook from top to bottom until you reach the section "Answer a Fairly Hard Question About the Data." After that, wait at the section "Answer a Fairly Hard Question About the Data." Then, follow the instructor's demonstration and enter your validated code as instructed. At the end of lecture, export your notebook to HTML and upload to the tAPP 5 assignment.**

## Assignment 

Complete the following steps:

### Personalize Your Notebook

1. **In this notebook file, **please fill in your name and your team's name** and save the notebook.**
2. **Rename your saved notebook with your own name.**

### Validate Your Input Data 

3. **Run the code cells below in order from top to bottom.**

Last time, we used a pretty vague shell glob to match our processed text input files with filenames like `names2000s.txt`. At some point we created some other files that could potentially match the shell glob and cause the bash interpreter to reduplicate some lines of data on the input. This is a form of data corruption, and we absolutely must not allow this to happen.  

To see if this might be happening to you, let's remember that we have 14 decades of data. Let's use UNIX text utilities to test how many files we are matching with our shell globs. In the following command, note that the option to `ls` is `-1` the number 1, not `-l` the letter l. This prints each file on a separate line. The backticks evaluate the pipeline to its output value.

4. **Run the following code cell with shift-enter.**

In [ ]:
if [ `ls -1 names*.txt | wc -l` -eq 14 ]; then 
    echo "CORRECT: 14 files matched with shell glob names*.txt" 
elif [ `ls -1 names*.txt | wc -l` -gt 14 ]; then 
    echo "INCORRECT: more than 14 files matched with shell glob names*.txt. Try a more specific shell glob."   
else 
    echo "INCORRECT: fewer than 14 files matched with shell glob names*.txt. You need to download or convert files."  
fi;


Let's try with a more specific shell glob that describes more exclusively our intended input file names for the decades from 1880s to 2010s. This can save the trouble of deleting files and let us proceed.

5. **Then run the following code cell with shift-enter.**

In [ ]:
if [ `ls -1 names[12][890][0-9]0s.txt | wc -l` -eq 14 ]; then 
    echo "CORRECT: 14 files matched with shell glob names[12][890][0-9]0s.txt " 
elif [ `ls -1 names[12][890][0-9]0s.txt | wc -l` -gt 14 ]; then 
    echo "INCORRECT: more than 14 files matched with shell glob names[12][890][0-9]0s.txt. Something's wrong, you need to delete some files. Try running ls -1 names[12][90][0-9]0s.txt in a code cell to see what's going on."   
else 
    echo "INCORRECT: fewer than 14 files matched with shell glob names[12][890][0-9]0s.txt. You need to download or convert some of the files from the data source."  
fi;

In [ ]:
ls -1 names[12][890][0-9]0s.txt


**The MORAL OF THE STORY with both shell globs and regular expressions is make them as specific and unambiguous as possible!**

### Download and Convert Source Data (Only if Necessary!)

**If the above two code chunks returned CORRECT, you may SKIP THIS SECTION to Step 7.**

6. **Run the following code cell with shift-enter.**

In [ ]:
pip install html2text
if [ `ls -1 names[12][890][0-9]0s.html | wc -l` -eq 14 ]; then 
    echo "14 data source input files in html detected. Converting files to text in Python."
    for decade in {188..201}0; do 
        html="names${decade}s.html"; 
        text="names${decade}s.txt"; 
        python -c 'import sys, html2text; print(html2text.html2text(sys.stdin.read()))' < $html > $text
    done;
else 
    echo "Downloading source data and converting files to text in Python."
    for decade in {188..201}0; do 
        html="names${decade}s.html"; 
        text="names${decade}s.txt"; 
        if [ -e $html ]; then
            echo "file $html exists. Skipping."
        else
            sleep 1; 
            wget --no-check-certificate "https://www.ssa.gov/oact/babynames/decades/names${decade}s.html"; 
            python -c 'import sys, html2text; print(html2text.html2text(sys.stdin.read()))' < $html > $text
        fi;
    done;
fi;

You may now re-run the tests under **Validate Your Input Data** above to validate that it shows "CORRECT"

### Extract Data Lines with Extended Bash Regular Expressions and Validate the Extraction (Only if Necessary!)

Last time we demonstrated the following POSIX-compliant extended regular expression in Bash `egrep` to match all of the 200 data lines from each of the 14 processed input data text files. We'll use `head` to just sample the first ten lines. Notice that when using `grep` or `egrep` with a shell glob, by default the output is labelled with file of origin.

Let's run some tests to validate that we are all extracting exactly the same number of lines and the same data.

7. **Run the following code cell with shift-enter.**

In [ ]:
echo "testing number of lines..."
if [ `grep -E "^[0-9]+ \|" names[12][890][0-9]0s.txt | wc -l` -eq 2800 ]; then 
    echo "2800 data lines detected. CORRECT."
else 
    echo "Fewer or Greater than 2800 data lines detected. INCORRECT. Amoungt found was:"
    grep -E "^[0-9]+ \|" names[12][890][0-9]0s.txt | wc -l
    echo "Investigate the output interactively in the cell below using wc, egrep, sort, cut, etc"
fi;
echo "testing equality of data..."
echo "...first ten lines..."
grep -E "^[0-9]+ \|" names[12][890][0-9]0s.txt | head
echo "...md5sum..."
grep -E "^[0-9]+ \|" names[12][890][0-9]0s.txt | head | md5sum
if [[ `grep -E "^[0-9]+ \|" names[12][890][0-9]0s.txt | head | md5sum` == "ddba814d0a3396d4bf85e9d59e6ada2b  -" ]]; then 
    echo "md5sum of first ten lines is CORRECT."
else 
    echo "ERROR: md5sum of first ten lines is INCORRECT."
fi;
echo "...all data..."
echo "...md5sum..."
grep -E "^[0-9]+ \|" names[12][890][0-9]0s.txt | md5sum
if [[ `grep -E "^[0-9]+ \|" names[12][890][0-9]0s.txt | md5sum` == "f110abd32f52d122b81119d08bf0c3ee  -" ]]; then 
    echo "md5sum of all data is CORRECT."
else 
    echo "ERROR: md5sum of all data is INCORRECT."
fi;

### Extract Data Lines with Perl Regular Expressions and Validate the Extraction

We'll see that `perl` is a powerful extension of the UNIX tools, replacing and superceding what `awk` and `sed` can do alone. Perl offers more powerful regular expression dialect too. In perl we can use `\d` instead of `[0-9]`. We can also use `\s` for white-space in general, matching both tabs and space charachters:

8. **Run the following code cell with shift-enter.**

In [ ]:
echo "testing number of lines with perl regexes using perl..."
if [ `perl -ne 'print if (/^\d+\s\|/)' names[12][890][0-9]0s.txt | wc -l` -eq 2800 ]; then 
    echo "2800 data lines detected. CORRECT."
else 
    echo "Fewer or Greater than 2800 data lines detected with perl. INCORRECT. Amoungt found was:"
    perl -ne 'print if (/^\d+\s\|/)' names[12][890][0-9]0s.txt | wc -l
    echo "Investigate the output interactively in the cell below using wc, egrep, sort, cut, etc"
fi;
echo "testing equality of data..."
echo "...first ten lines..."
perl -ne 'print "$ARGV:$_" if (/^\d+\s\|/)' names[12][890][0-9]0s.txt| head
echo "...md5sum..."
perl -ne 'print "$ARGV:$_" if (/^\d+\s\|/)' names[12][890][0-9]0s.txt| head | md5sum
if [[ `perl -ne 'print "$ARGV:$_" if (/^\d+\s\|/)' names[12][890][0-9]0s.txt| head | md5sum` == "ddba814d0a3396d4bf85e9d59e6ada2b  -" ]]; then 
    echo "md5sum of first ten lines is CORRECT."
else 
    echo "ERROR: md5sum of first ten lines is INCORRECT."
fi;
echo "...all data..."
echo "...md5sum..."
perl -ne 'print "$ARGV:$_" if (/^\d+\s\|/)' names[12][890][0-9]0s.txt| md5sum
if [[ `perl -ne 'print "$ARGV:$_" if (/^\d+\s\|/)' names[12][890][0-9]0s.txt| md5sum` == "f110abd32f52d122b81119d08bf0c3ee  -" ]]; then 
    echo "md5sum of all data is CORRECT."
else 
    echo "ERROR: md5sum of all data is INCORRECT."
fi;

## Clean Baby Names Data, Convert to CSV, and Validate (Only if Necessary!)

7. **Follow Instructor Deminstratino to Extract and Clean Data by editing the top of the code cell with a bash one-liner using a perl anonymous program for extraction.**
8. **Re-Run the following code cell with shift-enter to validate the one-liner.**

In [ ]:
perl -ne 'print "$ARGV:$_" if (/^\d+\s\|/)' names[12][890][0-9]0s.txt | cut -c 6-10,15- | tr -d " ," | tr ":|" , > baby_clean.csv
if [ `cat baby_clean.csv | wc -l` -eq 2800 ]; then 
    echo "2800 data lines detected in baby_clean.csv. CORRECT."
else 
    echo "Fewer or Greater than 2800 data lines detected in baby_clean.csv. INCORRECT. Amoungt found was:"
    cat baby_clean.csv | wc -l
    echo "Investigate the output interactively in the cell below using wc, egrep, sort, cut, etc"
fi;
echo "testing equality of data..."
echo "...first ten lines..."
head baby_clean.csv
echo "...md5sum..."
head baby_clean.csv | md5sum
if [[ `head baby_clean.csv | md5sum` == "6d8913fae285edec358d107db11dc4ed  -" ]]; then 
    echo "md5sum of first ten lines of baby_clean.csv is CORRECT."
else 
    echo "ERROR: md5sum of first ten lines of baby_clean.csv is INCORRECT."
fi;
echo "...all data..."
echo "...md5sum..."
cat baby_clean.csv | md5sum
if [[ `cat baby_clean.csv | md5sum` == "8fee16f6cd371c349dc6f30c04e4648c  -" ]]; then 
    echo "md5sum of all data in baby_clean.csv is CORRECT."
else 
    echo "ERROR: md5sum of all data in baby_clean.csv INCORRECT."
fi;

## Answer a Fairly Hard Question About the Data 

Write a single UNIX pipeline to answer the question (as an output number included in the notebook output you turn in): **how many unique boys and girls names are included in this dataset from all fourteen decades?** 

In [5]:
echo "Number of unique boy's and girl's names over all 2800 names in the top U.S. 200 most popular names by decade from 1880s to 2010s:"


Number of unique boy's and girl's names over all 2800 names in the top U.S. 200 most popular names by decade from 1880s to 2010s:


## Answer an Even Harder Question About the Data 

Write a single UNIX command pipeline to answer the question (as an output number included in the notebook output you turn in): **what is the frequency distribution of first letters among boy's names sorted by frequency from most common to least common?** 

In [ ]:
echo "Sorted Frequency Distribution of First Letters Among Top-Ranked Boy's Names over all 14 decades:"

## Use Python f-strings to Format the Number of Unique Names as a Percentage of the Total Number 

In [11]:
python -c 'print(f"{100 * ( / ):.2f}%")'

  File "<string>", line 1
    (100 * ( / ))
             ^
SyntaxError: f-string: invalid syntax


: 1

## Use Perl to Answer a Question that Cannot Be Answered with UNIX Text Utilities

While this first-letter frequency distribution over names is interesting, it doesn't predict well what a person who lived over these 14 decades would experiences as the first-letter distribution of first name's, because their actual experience of letters would be influenced by the **numbers of babies** born with each name. In order to compute this frequency distribution, we need to add up the numbers of babies born with each name. This is hard to do with UNIX text utilities because they don't treat strings of digits as numbers. Here is where Perl comes to the rescue.

**What is the frequency distribution of first letters among boy's names weighted by numbers of babies born with that name, sorted by frequencies per baby from most common to least common?** 

In [ ]:
echo "per baby:"
cut -d, -f baby_clean.csv | tr  | sort | perl -F, -ane '$ += $;END{foreach my $ (sort keys %){print ""}}'

## Use Perl to Check that the Number of Fields is the Same Over All Rows of Data

**How many fields are on every line of baby_clean.csv?** 

## Use Perl to Validate a Calculated Frequency Distribution

**What is the sum of first-letter frequencies of boy's names, is it equal to the total number of boy's names in the data?** 

In [ ]:
echo "sum of frequencies per name:"

**What is the sum of first-letter frequencies of boy's names weighted by numbers of births, is it equal to the total number of male babies in the data?** 

In [ ]:
echo "total male babies or sum of frequencies per baby:"
echo "total male babies in original source data:"
perl -ane 'print "" if (/^\d+\s\|/)' names[12][890][0-9]0s.txt | tr -d , | perl -ne ''

**Please export your worked notebook containing command output to HTML and upload it to CatCourses at the end of Lecture.**